In [1]:
import pandas as pd
import re

In [2]:
# Read .csv file
df_year_bc_extracted = pd.read_csv("E:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\df_year_bc_extracted.csv")

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_9248\2500825476.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_year_bc_extracted = pd.read_csv("E:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\df_year_bc_extracted.csv")


### Engine Size Formats in the DataFrame

There are two engine size formats present in the DataFrame:

1. **Engine size followed by 'L'**  
   Example: `2.0L`  
   - This format represents the engine displacement in liters, explicitly marked with the letter 'L'.

2. **Engine size without 'L', surrounded by whitespaces**  
   Example: ` 2.0 `  
   - This format represents the engine displacement in liters, but without the 'L' suffix.
   - The value is always surrounded by a whitespace before and after the number.

Both formats should be considered when extracting engine size information from the data.

In [3]:
def engine_size_extraction(df, pattern, new_col):

    # List to store all engine sizes
    engine_size_list =[]

    # Iterate over each line of the 'CALIBRATION' column
    for row in df['CALIBRATION']:
        # Condition to confirm whether the row is empty 
        # and when yes add empty data ("") to row, otherwise an error occurs.
        row_str = str(row) if not pd.isna(row) else ""
        # Extract all engine sizes based on the pattern
        engine_size = re.findall(pattern, row_str)

        # Condition when the engine_size variable is not empty
        if engine_size:
            engine_size_list.append(engine_size)
        else:
            # Add NA when 
            engine_size_list.append(pd.NA)

    # Update the main df with the engine sizes extracted
    # at a new column
    df[new_col] = engine_size_list

    # Converts each engine size from list to normal data
    df[new_col] = df[new_col].str.join('')

    # Return the df after engine size extraction
    return df

### **Engine size followed by 'L'**

In [4]:
# Pattern to capture engine sizes WITH L:
# \d -> one digit before .
# \. -> make sure . is literal on the regex
# \d -> one digit after .
# L -> letter L after last digit
engine_size_w_l_pattern = r'\d\.\dL'

In [5]:
# Call the function to extract the engine sizes with L
df_es_w_l_extraction = engine_size_extraction(df_year_bc_extracted, engine_size_w_l_pattern, "E.SL")

### **Engine size without 'L'**

In [6]:
# Pattern to capture engine sizes WITHOUT L:
# \s -> whitespaces before digit
# \d -> one digit (number)
# \. -> make the . literal
# \d -> another digit after .
# \s -> another whitespace after digit
engine_size_wo_l_pattern = r'\s\d\.\d\s'

In [7]:
# Call the function to extract the engine sizes without 
df_es_wo_l_extraction = engine_size_extraction(df_es_w_l_extraction, engine_size_wo_l_pattern, "E.S") 

In [8]:
# List to append the engine sizes after adding l
add_l_list = []

# For to iterate over the items of col 'E.S'
for engine in df_es_wo_l_extraction["E.S"]:
    # Condition to confirm when the item is engine size 2.0
    if not pd.isna(engine) and len(str(engine)) > 4:
        # Remove the whitespace and add L to become 2.0L
        add_l_list.append(f"{engine.strip()}" + "L")
    else:
        add_l_list.append(engine)

In [9]:
# Update the 'E.S' after adding L to the engine sizes
df_es_wo_l_extraction["E.S"] = add_l_list

In [10]:
# Fill out null values at column E.SL with engine sizes extracted from column E.S
df_es_wo_l_extraction["E.SL"] = df_es_wo_l_extraction["E.SL"].fillna(df_es_wo_l_extraction["E.S"])

In [11]:
# Drop column E.S
df_es_extraction_drop_es_col = df_es_wo_l_extraction.drop(columns=["E.S"])

In [12]:
# Rename engine size col
df_es_extraction_rn_col = df_es_extraction_drop_es_col.rename(columns={"E.SL": "Engine Size"})

In [13]:
# Check df info
df_es_extraction_rn_col.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15589 entries, 0 to 15588
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   CALIBRATION         15586 non-null  object
 1   Year                15577 non-null  object
 2   TYPE                15586 non-null  object
 3   OLD PART NUMBER(S)  15124 non-null  object
 4   NEW PART NUMBER(s)  15586 non-null  object
 5   Body                15584 non-null  object
 6   Engine Size         15324 non-null  object
dtypes: object(7)
memory usage: 852.7+ KB


In [14]:
# Replace line break '\n' and keep the part numbers in the same line
df_es_extraction_rn_col['OLD PART NUMBER(S)'] = df_es_extraction_rn_col['OLD PART NUMBER(S)'].str.replace("\n", ", ")

In [15]:
# Export to .csv file
df_es_extraction_rn_col.to_csv("E:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\df_year_bc_es_extracted.csv", index=False)

<>:2: SyntaxWarning: invalid escape sequence '\L'
<>:2: SyntaxWarning: invalid escape sequence '\L'
C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_9248\1839590293.py:2: SyntaxWarning: invalid escape sequence '\L'
  df_es_extraction_rn_col.to_csv("E:\Language_Projects\Language_Projects\Python\Flagship_1\pdf_data_extract\data\df_year_bc_es_extracted.csv", index=False)
